In [1]:
import numpy as np

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("final_training.csv")
df

Residue_Index Amino_Acid   PDBID Secondary_Structure Junction_Region  \
0                  1          A  1DZI.A                   C               C   
1                  2          L  1DZI.A                   C               C   
2                  3          I  1DZI.A                   C               J   
3                  4          D  1DZI.A                   E               J   
4                  5          V  1DZI.A                   E               E   
...              ...        ...     ...                 ...             ...   
17990             84          G  5E6P.B                   C               C   
17991             85          S  5E6P.B                   C               C   
17992             86          S  5E6P.B                   C               C   
17993             87          P  5E6P.B                   C               C   
17994             88          S  5E6P.B                   C               C   

       Arginine_Propensity  Tryptophan_Propensity  Tyrosine_Propensity  \
0                      0.0                    0.0                  0.0   
1                      0.0                    0.0                  0.0   
2                      0.0                    0.0                  0.0   
3                      0.0                    0.0                  0.0   
4                      0.0                    0.0                  0.0   
...                    ...                    ...                  ...   
17990                  0.0                    0.0                  0.0   
17991                  0.0                    0.0                  0.0   
17992                  0.0                    0.0                  0.0   
17993                  0.0                    0.0                  0.0   
17994                  0.0                    0.0                  0.0   

       Valine_Propensity  Serine_Propensity  Methionine_Propensity  \
0               0.000000           0.000000                    0.0   
1               0.000000           0.000000                    0.0   
2               0.000000           0.000000                    0.0   
3               0.428571           0.000000                    0.0   
4               0.571429           0.000000                    0.0   
...                  ...                ...                    ...   
17990           0.000000           0.285714                    0.0   
17991           0.000000           0.428571                    0.0   
17992           0.000000           0.500000                    0.0   
17993           0.000000           0.600000                    0.0   
17994           0.000000           0.750000                    0.0   

       Threonine_Propensity  Leucine_Propensity Chain   RelSASA  \
0                  0.000000            0.000000     A  0.738996   
1                  0.000000            0.000000     A  0.417377   
2                  0.000000            0.000000     A  0.110747   
3                  0.000000            0.142857     A  0.094342   
4                  0.000000            0.142857     A  0.016116   
...                     ...                 ...   ...       ...   
17990              0.142857            0.285714     B  0.424894   
17991              0.000000            0.285714     B  0.603341   
17992              0.000000            0.166667     B  0.689852   
17993              0.000000            0.000000     B  0.742278   
17994              0.000000            0.000000     B  0.787751   

       Secondary_Enum  Label  
0            0.433300    NaN  
1            0.433300    NaN  
2            0.942029    NaN  
3            0.942029    NaN  
4            0.171340    NaN  
...               ...    ...  
17990        0.433300    NaN  
17991        0.433300    1.0  
17992        0.433300    1.0  
17993        0.433300    1.0  
17994        0.433300    1.0  

[17995 rows x 17 columns]

In [3]:
df['Label'] = df['Label'].replace(np.nan, 0)
df

Residue_Index Amino_Acid   PDBID Secondary_Structure Junction_Region  \
0                  1          A  1DZI.A                   C               C   
1                  2          L  1DZI.A                   C               C   
2                  3          I  1DZI.A                   C               J   
3                  4          D  1DZI.A                   E               J   
4                  5          V  1DZI.A                   E               E   
...              ...        ...     ...                 ...             ...   
17990             84          G  5E6P.B                   C               C   
17991             85          S  5E6P.B                   C               C   
17992             86          S  5E6P.B                   C               C   
17993             87          P  5E6P.B                   C               C   
17994             88          S  5E6P.B                   C               C   

       Arginine_Propensity  Tryptophan_Propensity  Tyrosine_Propensity  \
0                      0.0                    0.0                  0.0   
1                      0.0                    0.0                  0.0   
2                      0.0                    0.0                  0.0   
3                      0.0                    0.0                  0.0   
4                      0.0                    0.0                  0.0   
...                    ...                    ...                  ...   
17990                  0.0                    0.0                  0.0   
17991                  0.0                    0.0                  0.0   
17992                  0.0                    0.0                  0.0   
17993                  0.0                    0.0                  0.0   
17994                  0.0                    0.0                  0.0   

       Valine_Propensity  Serine_Propensity  Methionine_Propensity  \
0               0.000000           0.000000                    0.0   
1               0.000000           0.000000                    0.0   
2               0.000000           0.000000                    0.0   
3               0.428571           0.000000                    0.0   
4               0.571429           0.000000                    0.0   
...                  ...                ...                    ...   
17990           0.000000           0.285714                    0.0   
17991           0.000000           0.428571                    0.0   
17992           0.000000           0.500000                    0.0   
17993           0.000000           0.600000                    0.0   
17994           0.000000           0.750000                    0.0   

       Threonine_Propensity  Leucine_Propensity Chain   RelSASA  \
0                  0.000000            0.000000     A  0.738996   
1                  0.000000            0.000000     A  0.417377   
2                  0.000000            0.000000     A  0.110747   
3                  0.000000            0.142857     A  0.094342   
4                  0.000000            0.142857     A  0.016116   
...                     ...                 ...   ...       ...   
17990              0.142857            0.285714     B  0.424894   
17991              0.000000            0.285714     B  0.603341   
17992              0.000000            0.166667     B  0.689852   
17993              0.000000            0.000000     B  0.742278   
17994              0.000000            0.000000     B  0.787751   

       Secondary_Enum  Label  
0            0.433300    0.0  
1            0.433300    0.0  
2            0.942029    0.0  
3            0.942029    0.0  
4            0.171340    0.0  
...               ...    ...  
17990        0.433300    0.0  
17991        0.433300    1.0  
17992        0.433300    1.0  
17993        0.433300    1.0  
17994        0.433300    1.0  

[17995 rows x 17 columns]

In [4]:
aa_to_number = {
    'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5,
    'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10,
    'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15,
    'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20
}

# Convert Amino_Acid column to numbers
df['Amino_Acid'] = df['Amino_Acid'].map(aa_to_number)
df

Residue_Index  Amino_Acid   PDBID Secondary_Structure Junction_Region  \
0                  1           1  1DZI.A                   C               C   
1                  2          10  1DZI.A                   C               C   
2                  3           8  1DZI.A                   C               J   
3                  4           3  1DZI.A                   E               J   
4                  5          18  1DZI.A                   E               E   
...              ...         ...     ...                 ...             ...   
17990             84           6  5E6P.B                   C               C   
17991             85          16  5E6P.B                   C               C   
17992             86          16  5E6P.B                   C               C   
17993             87          13  5E6P.B                   C               C   
17994             88          16  5E6P.B                   C               C   

       Arginine_Propensity  Tryptophan_Propensity  Tyrosine_Propensity  \
0                      0.0                    0.0                  0.0   
1                      0.0                    0.0                  0.0   
2                      0.0                    0.0                  0.0   
3                      0.0                    0.0                  0.0   
4                      0.0                    0.0                  0.0   
...                    ...                    ...                  ...   
17990                  0.0                    0.0                  0.0   
17991                  0.0                    0.0                  0.0   
17992                  0.0                    0.0                  0.0   
17993                  0.0                    0.0                  0.0   
17994                  0.0                    0.0                  0.0   

       Valine_Propensity  Serine_Propensity  Methionine_Propensity  \
0               0.000000           0.000000                    0.0   
1               0.000000           0.000000                    0.0   
2               0.000000           0.000000                    0.0   
3               0.428571           0.000000                    0.0   
4               0.571429           0.000000                    0.0   
...                  ...                ...                    ...   
17990           0.000000           0.285714                    0.0   
17991           0.000000           0.428571                    0.0   
17992           0.000000           0.500000                    0.0   
17993           0.000000           0.600000                    0.0   
17994           0.000000           0.750000                    0.0   

       Threonine_Propensity  Leucine_Propensity Chain   RelSASA  \
0                  0.000000            0.000000     A  0.738996   
1                  0.000000            0.000000     A  0.417377   
2                  0.000000            0.000000     A  0.110747   
3                  0.000000            0.142857     A  0.094342   
4                  0.000000            0.142857     A  0.016116   
...                     ...                 ...   ...       ...   
17990              0.142857            0.285714     B  0.424894   
17991              0.000000            0.285714     B  0.603341   
17992              0.000000            0.166667     B  0.689852   
17993              0.000000            0.000000     B  0.742278   
17994              0.000000            0.000000     B  0.787751   

       Secondary_Enum  Label  
0            0.433300    0.0  
1            0.433300    0.0  
2            0.942029    0.0  
3            0.942029    0.0  
4            0.171340    0.0  
...               ...    ...  
17990        0.433300    0.0  
17991        0.433300    1.0  
17992        0.433300    1.0  
17993        0.433300    1.0  
17994        0.433300    1.0  

[17995 rows x 17 columns]

In [5]:
df["Valine_Propensity"]=df["Valine_Propensity"].apply(lambda x:x*-1)
df["Serine_Propensity"]=df["Serine_Propensity"].apply(lambda x:x*-1)
df["Methionine_Propensity"]=df["Methionine_Propensity"].apply(lambda x:x*-1)
df["Threonine_Propensity"]=df["Threonine_Propensity"].apply(lambda x:x*-1)
df["Leucine_Propensity"]=df["Leucine_Propensity"].apply(lambda x:x*-1)
df

Residue_Index  Amino_Acid   PDBID Secondary_Structure Junction_Region  \
0                  1           1  1DZI.A                   C               C   
1                  2          10  1DZI.A                   C               C   
2                  3           8  1DZI.A                   C               J   
3                  4           3  1DZI.A                   E               J   
4                  5          18  1DZI.A                   E               E   
...              ...         ...     ...                 ...             ...   
17990             84           6  5E6P.B                   C               C   
17991             85          16  5E6P.B                   C               C   
17992             86          16  5E6P.B                   C               C   
17993             87          13  5E6P.B                   C               C   
17994             88          16  5E6P.B                   C               C   

       Arginine_Propensity  Tryptophan_Propensity  Tyrosine_Propensity  \
0                      0.0                    0.0                  0.0   
1                      0.0                    0.0                  0.0   
2                      0.0                    0.0                  0.0   
3                      0.0                    0.0                  0.0   
4                      0.0                    0.0                  0.0   
...                    ...                    ...                  ...   
17990                  0.0                    0.0                  0.0   
17991                  0.0                    0.0                  0.0   
17992                  0.0                    0.0                  0.0   
17993                  0.0                    0.0                  0.0   
17994                  0.0                    0.0                  0.0   

       Valine_Propensity  Serine_Propensity  Methionine_Propensity  \
0              -0.000000          -0.000000                   -0.0   
1              -0.000000          -0.000000                   -0.0   
2              -0.000000          -0.000000                   -0.0   
3              -0.428571          -0.000000                   -0.0   
4              -0.571429          -0.000000                   -0.0   
...                  ...                ...                    ...   
17990          -0.000000          -0.285714                   -0.0   
17991          -0.000000          -0.428571                   -0.0   
17992          -0.000000          -0.500000                   -0.0   
17993          -0.000000          -0.600000                   -0.0   
17994          -0.000000          -0.750000                   -0.0   

       Threonine_Propensity  Leucine_Propensity Chain   RelSASA  \
0                 -0.000000           -0.000000     A  0.738996   
1                 -0.000000           -0.000000     A  0.417377   
2                 -0.000000           -0.000000     A  0.110747   
3                 -0.000000           -0.142857     A  0.094342   
4                 -0.000000           -0.142857     A  0.016116   
...                     ...                 ...   ...       ...   
17990             -0.142857           -0.285714     B  0.424894   
17991             -0.000000           -0.285714     B  0.603341   
17992             -0.000000           -0.166667     B  0.689852   
17993             -0.000000           -0.000000     B  0.742278   
17994             -0.000000           -0.000000     B  0.787751   

       Secondary_Enum  Label  
0            0.433300    0.0  
1            0.433300    0.0  
2            0.942029    0.0  
3            0.942029    0.0  
4            0.171340    0.0  
...               ...    ...  
17990        0.433300    0.0  
17991        0.433300    1.0  
17992        0.433300    1.0  
17993        0.433300    1.0  
17994        0.433300    1.0  

[17995 rows x 17 columns]

In [6]:
label="PDBID"
classes= df[label].unique().tolist()
print(f"Label classes: {classes}")
df[label]= df[label].map(classes.index)

Label classes: ['1DZI.A', '1JTG.A', '1JTG.B', '1CDL.A', '2WPT.B', '4X4M.E', '1EAW.A', '1BGS.E', '2HHB.D', '1N8Z.B', '1JAT.B', '4YFD.B', '1DDM.B', '1AXI.A', '4HMY.C', '1BGS.A', '4U6H.E', '4NM8.B', '1F47.B', '1BRS.D', '1EBP.C', '3NPS.A', '1ES7.A', '2KSO.A', '4FZA.B', '1A22.A', '3Q3J.B', '1A4Y.B', '2NZ9.A', '3BP8.A', '4KRM.B', '3NGB.C', '1A4Y.A', '1DVF.B', '1FCC.C', '1JAT.A', '1K8R.A', '1KTZ.A', '1KTZ.B', '1MLC.B', '1REW.A', '1REW.B', '1REW.C', '1VFB.A', '1VFB.C', '2G2U.B', '2PCC.A', '2X0B.B', '3BN9.B', '3HHR.A', '3K2M.D', '3MBW.A', '3SAK.A', '3W2D.A', '4IOF.A', '4J2L.C', '4JEU.A', '4L72.A', '4LRX.A', '4O27.B', '4OZG.F', '4P23.C', '4PWX.C', '4UWQ.A', '4Y61.A', '4YEB.A', '5C6T.A', '5CYK.A', '5E6P.A', '5E6P.B']


In [7]:
X = df.drop(["Label", "Residue_Index","PDBID","Chain","Secondary_Structure","Junction_Region"], axis=1)
y = df["Label"]

In [8]:
X

Amino_Acid  Arginine_Propensity  Tryptophan_Propensity  \
0               1                  0.0                    0.0   
1              10                  0.0                    0.0   
2               8                  0.0                    0.0   
3               3                  0.0                    0.0   
4              18                  0.0                    0.0   
...           ...                  ...                    ...   
17990           6                  0.0                    0.0   
17991          16                  0.0                    0.0   
17992          16                  0.0                    0.0   
17993          13                  0.0                    0.0   
17994          16                  0.0                    0.0   

       Tyrosine_Propensity  Valine_Propensity  Serine_Propensity  \
0                      0.0          -0.000000          -0.000000   
1                      0.0          -0.000000          -0.000000   
2                      0.0          -0.000000          -0.000000   
3                      0.0          -0.428571          -0.000000   
4                      0.0          -0.571429          -0.000000   
...                    ...                ...                ...   
17990                  0.0          -0.000000          -0.285714   
17991                  0.0          -0.000000          -0.428571   
17992                  0.0          -0.000000          -0.500000   
17993                  0.0          -0.000000          -0.600000   
17994                  0.0          -0.000000          -0.750000   

       Methionine_Propensity  Threonine_Propensity  Leucine_Propensity  \
0                       -0.0             -0.000000           -0.000000   
1                       -0.0             -0.000000           -0.000000   
2                       -0.0             -0.000000           -0.000000   
3                       -0.0             -0.000000           -0.142857   
4                       -0.0             -0.000000           -0.142857   
...                      ...                   ...                 ...   
17990                   -0.0             -0.142857           -0.285714   
17991                   -0.0             -0.000000           -0.285714   
17992                   -0.0             -0.000000           -0.166667   
17993                   -0.0             -0.000000           -0.000000   
17994                   -0.0             -0.000000           -0.000000   

        RelSASA  Secondary_Enum  
0      0.738996        0.433300  
1      0.417377        0.433300  
2      0.110747        0.942029  
3      0.094342        0.942029  
4      0.016116        0.171340  
...         ...             ...  
17990  0.424894        0.433300  
17991  0.603341        0.433300  
17992  0.689852        0.433300  
17993  0.742278        0.433300  
17994  0.787751        0.433300  

[17995 rows x 11 columns]

In [9]:
# import XGBoost
import xgboost as xgb

# # define data_dmatrix
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [10]:
# split X and y into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [11]:
# Calculate the ratio of negative class to positive class
ratio = float(y_train.value_counts()[0]) / y_train.value_counts()[1]
print(ratio)

8.727027027027027


In [12]:
# import XGBClassifier
from xgboost import XGBClassifier


# # declare parameters
# params = {
#             'objective':'binary:logistic',
#             'max_depth': 4,
#             'alpha': 10,
#             'learning_rate': 1.0,
#             'n_estimators':100
#         }



# instantiate the classifier
xgb_clf = XGBClassifier(learning_rate =0.1,
 n_estimators=100,
 max_depth=45,
 min_child_weight=1,
 gamma=0,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=13)


# without scale pos weight or SMOTE
# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=45, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, nthread=4,
              num_parallel_tree=None, ...)

In [13]:
# make predictions on test data
y_pred = xgb_clf.predict(X_test)

In [14]:
# check accuracy score
from sklearn.metrics import accuracy_score

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

XGBoost model accuracy score: 0.8955
              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94      3229
         1.0       0.47      0.13      0.20       370

    accuracy                           0.90      3599
   macro avg       0.69      0.56      0.57      3599
weighted avg       0.86      0.90      0.87      3599



In [17]:
#with cale pos weight
model = xgb.XGBClassifier(scale_pos_weight=ratio)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

XGBoost model accuracy score: 0.7410
              precision    recall  f1-score   support

         0.0       0.92      0.78      0.84      3229
         1.0       0.17      0.38      0.23       370

    accuracy                           0.74      3599
   macro avg       0.54      0.58      0.54      3599
weighted avg       0.84      0.74      0.78      3599



In [19]:
#with SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 20)

# Fit predictor (x variable)
# and target (y variable) using fit_resample()
X_resampled, y_resampled = sm.fit_resample(X, y)
#USE THIS MODEL FOR EXTRACTING COEFFICIENTS


# Splitting dataset
train_X, test_X, train_y, test_y = train_test_split(X_resampled, y_resampled, test_size = 0.20)

# Instantiate Gradient Boosting Regressor
xg = XGBClassifier(learning_rate =0.1,
 n_estimators=100,
 max_depth=45,
 min_child_weight=1,
 gamma=0,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=13)
xg.fit(train_X, train_y)

# Predict on test set
pred_y = xg.predict(test_X)

# accuracy
acc = accuracy_score(test_y, pred_y)
print("xgboost with SMOTE accuracy is : {:.2f}".format(acc))
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_y))

xgboost with SMOTE accuracy is : 0.93
              precision    recall  f1-score   support

         0.0       0.91      0.96      0.94      3222
         1.0       0.96      0.91      0.93      3236

    accuracy                           0.93      6458
   macro avg       0.94      0.93      0.93      6458
weighted avg       0.94      0.93      0.93      6458



In [20]:
df1 = pd.read_excel("viral-ml.xlsx")
df1

Residue_Index PDBID Amino_Acid  Arginine_Propensity  \
0                 1   PB2          M             0.000000   
1                 2   PB2          E             0.000000   
2                 3   PB2          R             0.000000   
3                 4   PB2          I             0.142857   
4                 5   PB2          K             0.285714   
...             ...   ...        ...                  ...   
3317            116   NEP          S             0.142857   
3318            117   NEP          F             0.000000   
3319            118   NEP          Q             0.000000   
3320            119   NEP          L             0.000000   
3321            120   NEP          I             0.000000   

      Tryptophan_Propensity  Tyrosine_Propensity  Valine_Propensity  \
0                       0.0                  0.0                0.0   
1                       0.0                  0.0                0.0   
2                       0.0                  0.0                0.0   
3                       0.0                  0.0                0.0   
4                       0.0                  0.0                0.0   
...                     ...                  ...                ...   
3317                    0.0                  0.0                0.0   
3318                    0.0                  0.0                0.0   
3319                    0.0                  0.0                0.0   
3320                    0.0                  0.0                0.0   
3321                    0.0                  0.0                0.0   

      Serine_Propensity  Methionine_Propensity  Threonine_Propensity  \
0              0.000000               0.000000                   0.0   
1              0.000000               0.000000                   0.0   
2              0.000000               0.000000                   0.0   
3              0.000000               0.142857                   0.0   
4              0.000000               0.000000                   0.0   
...                 ...                    ...                   ...   
3317           0.142857               0.000000                   0.0   
3318           0.142857               0.000000                   0.0   
3319           0.000000               0.000000                   0.0   
3320           0.000000               0.000000                   0.0   
3321           0.000000               0.000000                   0.0   

      Leucine_Propensity   RelSASA  Secondary_Enum  
0               0.000000  0.783568        0.942029  
1               0.000000  0.742773        0.942029  
2               0.000000  0.592849        0.130701  
3               0.142857  0.516733        0.130701  
4               0.142857  0.942029        0.130701  
...                  ...       ...             ...  
3317            0.142857  0.435100        0.276250  
3318            0.142857  0.731220        0.276250  
3319            0.000000  0.695062        0.942029  
3320            0.000000  0.942029        0.942029  
3321            0.000000  0.837853        0.460070  

[3322 rows x 13 columns]

In [22]:
df1=df1.drop(["Residue_Index", "PDBID"], axis=1)

In [23]:
df1['Amino_Acid'] = df1['Amino_Acid'].map(aa_to_number)

In [24]:
# label="PDBID"
# classes= df1[label].unique().tolist()
# print(f"Label classes: {classes}")
# df1[label]= df1[label].map(classes.index)


In [25]:
df1["Valine_Propensity"]=df1["Valine_Propensity"].apply(lambda x:x*-1)
df1["Serine_Propensity"]=df1["Serine_Propensity"].apply(lambda x:x*-1)
df1["Methionine_Propensity"]=df1["Methionine_Propensity"].apply(lambda x:x*-1)
df1["Threonine_Propensity"]=df1["Threonine_Propensity"].apply(lambda x:x*-1)
df1["Leucine_Propensity"]=df1["Leucine_Propensity"].apply(lambda x:x*-1)
df1

Amino_Acid  Arginine_Propensity  Tryptophan_Propensity  \
0             11             0.000000                    0.0   
1              4             0.000000                    0.0   
2             15             0.000000                    0.0   
3              8             0.142857                    0.0   
4              9             0.285714                    0.0   
...          ...                  ...                    ...   
3317          16             0.142857                    0.0   
3318           5             0.000000                    0.0   
3319          14             0.000000                    0.0   
3320          10             0.000000                    0.0   
3321           8             0.000000                    0.0   

      Tyrosine_Propensity  Valine_Propensity  Serine_Propensity  \
0                     0.0               -0.0          -0.000000   
1                     0.0               -0.0          -0.000000   
2                     0.0               -0.0          -0.000000   
3                     0.0               -0.0          -0.000000   
4                     0.0               -0.0          -0.000000   
...                   ...                ...                ...   
3317                  0.0               -0.0          -0.142857   
3318                  0.0               -0.0          -0.142857   
3319                  0.0               -0.0          -0.000000   
3320                  0.0               -0.0          -0.000000   
3321                  0.0               -0.0          -0.000000   

      Methionine_Propensity  Threonine_Propensity  Leucine_Propensity  \
0                 -0.000000                  -0.0           -0.000000   
1                 -0.000000                  -0.0           -0.000000   
2                 -0.000000                  -0.0           -0.000000   
3                 -0.142857                  -0.0           -0.142857   
4                 -0.000000                  -0.0           -0.142857   
...                     ...                   ...                 ...   
3317              -0.000000                  -0.0           -0.142857   
3318              -0.000000                  -0.0           -0.142857   
3319              -0.000000                  -0.0           -0.000000   
3320              -0.000000                  -0.0           -0.000000   
3321              -0.000000                  -0.0           -0.000000   

       RelSASA  Secondary_Enum  
0     0.783568        0.942029  
1     0.742773        0.942029  
2     0.592849        0.130701  
3     0.516733        0.130701  
4     0.942029        0.130701  
...        ...             ...  
3317  0.435100        0.276250  
3318  0.731220        0.276250  
3319  0.695062        0.942029  
3320  0.942029        0.942029  
3321  0.837853        0.460070  

[3322 rows x 11 columns]

In [26]:
predictions1 = xg.predict(df1)

In [ ]:
# csv_file_path = "preds_scaleposw.csv"

# # Create a DataFrame with the predicted labels
# result_df = pd.DataFrame({'Predicted_Labels': predictions1.flatten()})

# # Save the DataFrame to a CSV file
# result_df.to_csv(csv_file_path, index=False)

# # Print a message indicating the successful save
# print(f"Predictions saved to {csv_file_path}")

In [27]:
df2 = pd.read_excel("external-val.xlsx")
df2['LUBNA'] = df2['LUBNA'].replace(np.nan, 0)

In [28]:
test=df2['LUBNA']

In [29]:
print(classification_report(test, predictions1))

              precision    recall  f1-score   support

         0.0       0.58      0.95      0.72      1927
         1.0       0.39      0.04      0.08      1395

    accuracy                           0.57      3322
   macro avg       0.49      0.50      0.40      3322
weighted avg       0.50      0.57      0.45      3322



In [ ]:
# bst = model.get_booster()
# bst.dump_model('dump.raw.txt')

In [ ]:
# model.get_booster().trees_to_dataframe()